In [1]:
import json
import os
import pandas as pd
import random
import sys
from datetime import date
from dotenv import load_dotenv
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils.utils import GPT
import utils.utils as utils

In [2]:
# params
# st_mdl = "gpt-3.5-turbo-1106" # model string
st_mdl = "gpt-4-1106-preview"
st_task = "csqa" # name of task

# construct filenames
st_today = date.today().strftime("%Y_%m_%d") # today's date
st_fn = f"{st_today}_{st_task}_{st_mdl}" # file name
curr_dir = os.path.dirname(os.path.abspath("__file__"))

logger = utils.config_logger(curr_dir, st_fn)

In [3]:
# load api key from .env and create GPT object
load_dotenv()  # Load environment variables from .env file
openai_api_key = os.getenv("OPENAI_API_KEY")
gpt = GPT(openai_api_key, logger=logger, model=st_mdl)

# Load into a dictionary
# with open('data/24game.csv', 'r') as f:
#     data = json.load(f) # keys in each row: ['_id', 'answer', 'question', 'supporting_facts', 'context', 'type', 'level']

data = []
dir_data = 'data'
for dir_cat in os.listdir(dir_data): # category dir
    path_cat = os.path.join(dir_data, dir_cat)
    if os.path.isdir(path_cat):
        for fn in os.listdir(path_cat):
            if fn.endswith('.json'):
                fp = os.path.join(path_cat, fn)
                with open(fp, 'r') as f:
                    json_data = json.load(f)
                if len(json_data) >= 2:
                    data.append([json_data[0], json_data[1]])

print(len(data))

# data = pd.read_csv('data/24game.csv').to_dict(orient='records') # load CSV into list of dicts

27772


In [4]:
formatted_data = []
for idx, task in enumerate(data):
    if task[0]['speaker'] == 'USER' and task[1]['speaker'] == 'SYSTEM':
        q = task[0]['utterance']
        a = task[1]['utterance']
        formatted_data.append({'_id': idx, 'question': q, 'answer': a})
        
print(len(formatted_data))
data = formatted_data

27772


In [5]:
def compare_strings(str1, str2):
    """
    This function compares two strings after converting them to lowercase and stripping leading and trailing spaces.
    """
    return str1.lower().strip() == str(str2).lower().strip()

def check_bool(str1, bool2):
    """
        Checks if string str1 equal to bool bool2
    """
    try:
        if bool(str1) == bool2:
            return True
    except:
        pass

    return False
        

In [6]:
# dataset-specific adjustments (we eventually want a list of dicts of format [{"id": "xx", "question": "xxx", "context": "xxx"}, ...]

# pre-process data

df = pd.json_normalize(data)
df.head()

,_id,question,answer
0,0,Which administrative territory is the country ...,United States of America
1,1,Which occupations have Ben Clark as his/her 's...,"translator, writer, poet"
2,2,Which award was received by Poilly ?,Croix de guerre 1914–1918
3,3,What is the sex of Christian Bach ?,female
4,4,Which magazine does François Gautier work for ?,Marianne


## Generator

In [7]:
predictions = [] # list of predictions, reset after each set of runs
indices = [] # question numbers
evals = [] # whether prediction matches answer
questions = [] # list of questions
answers = [] # list of answers


# Evaluate
num_questions = 200 # number of questions to run on
indices = random.sample(range(len(data)), num_questions)
for idx in indices:
    item = data[idx]
    gold, q = item['answer'], item['question']
    logger.info(q)
    
    msg = [{"role": "system", "content": "Answer in a single noun, verb, or adjective"}, {"role": "user", "content": q}]
    predictions.append(gpt.gen(msgs=msg).content)
    answers.append(gold)
    evals.append(compare_strings(predictions[-1], gold))
    questions.append(q)
    print(predictions[-1])

# Print the predictions and answers
logger.info(predictions)
# logger.info(answers)
percent_correct = (sum(evals) / len(evals)) * 100

print(f"Percentage of True values: {percent_correct}%")

# create a dataframe with the data
df = pd.DataFrame({'Index': indices, 'Question': questions, 'Prediction': predictions, 'Answer': answers, 'Eval': evals})

# save the datafrme to csv
df.to_csv(f'results/{st_fn}.csv', index=False)

2024-02-14 09:20:51,046 - INFO - What are a biological processes that Gastric triacylglycerol lipase are involved in ?
2024-02-14 09:20:51,566 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 09:20:51,569 - INFO - Which occupations have Jacob Jervell as his/her 's career ?


Digestion


2024-02-14 09:20:52,518 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 09:20:52,520 - INFO - Where was Quills filmed ?


Theologian


2024-02-14 09:20:52,951 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 09:20:52,953 - INFO - What encodes sensory histidine kinase in two-component regulatory system with citB    ECIAI39_0595 ?


...

SRP9


2024-02-14 09:23:28,870 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 09:23:28,871 - INFO - Which occupation was a position held by Antoine-François Hardy ?


Football


2024-02-14 09:23:32,013 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 09:23:32,015 - INFO - ['Digestion', 'Theologian', 'Australia', 'Gene', 'Cycling', 'Botanist', 'Animator', 'unknown', 'Schwerin', 'Cancer', 'Spermatogenesis', 'Pre2', 'Gironde', 'Hebrew', 'Economist', 'Documentarists', 'Director', 'Architect.', 'Bounty-hunter', 'Unknown', 'France', 'Catholicism', 'Cytosol', 'Historian', 'Unknown', 'Chris', 'Sorry, I cannot provide current employment information for specific individuals unless it is widely known and publicly available information about a prominent person in society.', "I'm sorry, but without specific information about who Kofi Mensah is, I cannot provide an answer. There could be many people with the name Kofi Mensah from different administrative territories. If you can provide additional context or details about this person, I may be able to assist you better.", 'battle', 'Enzyme', 'unknown', 'Academician', 'Sicily

Cartographer
Percentage of True values: 13.5%


## Validator

In [8]:
gen_data_fn = f"results/{st_fn}.csv" # change this to the filename of the generator results
# gen_data_fn = 'results/2024_01_24_n200_gpt-3.5-turbo-1106_with_sols.csv'

df = pd.read_csv(gen_data_fn)
indices = df.iloc[:, 0].tolist()
indices = indices[:-1] # just because we manually added an additional row on the CSV when calculating scores

val_preds = [] # list of predictions for validator, reset after each set of runs
val_evals = [] # whether prediction matches answer


# Evaluate
for _, row in df.iterrows():
    idx = row['Index']
    ans = row['Prediction']
    gold = row['Manual Eval']
    
    # question, context = item['question'], item['context']
    q = "Input: " + row['Question'] + " "
    q += f"Answer: {ans}"
    logger.info(q)
    msg = [{"role": "system", "content": "Answer 'TRUE' if the answer is correct and 'FALSE' if the answer is incorrect."}, {"role": "user", "content": q}]
    val_preds.append(gpt.gen(msgs=msg).content)
    val_evals.append(check_bool(val_preds[-1], gold))
    print(val_preds[-1])

# Print the predictions and answers
logger.info(val_preds)
logger.info(df['Eval'].tolist())
percent_correct = (sum(val_evals) / len(val_evals)) * 100

print(f"Percentage of True values: {percent_correct}%")

# add validator predictions to column
df['Validator Pred'] = val_preds

# save the dataframe to csv
df.to_csv(f'results/{st_fn}_with_val.csv', index=False);

2024-02-14 09:36:11,227 - INFO - Input: What are a biological processes that Gastric triacylglycerol lipase are involved in ? Answer: Digestion
2024-02-14 09:36:12,008 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 09:36:12,010 - INFO - Input: Which occupations have Jacob Jervell as his/her 's career ? Answer: Theologian


TRUE


2024-02-14 09:36:12,653 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 09:36:12,655 - INFO - Input: Where was Quills filmed ? Answer: Australia


TRUE


2024-02-14 09:36:13,061 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 09:36:13,063 - INFO - Input: What encodes sensory histidine kinase in two-component regulatory system with citB    ECIAI39_0595 ? Answer: Gene


...

TRUE


2024-02-14 09:39:16,700 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 09:39:16,702 - INFO - Input: Which occupation was a position held by Antoine-François Hardy ? Answer: Cartographer


TRUE


2024-02-14 09:39:23,730 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 502 Bad Gateway"
2024-02-14 09:39:23,731 - INFO - Retrying request to /chat/completions in 0.853931 seconds
2024-02-14 09:39:25,058 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 09:39:25,060 - INFO - ['TRUE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE\n\n(Without specific information regarding an individual named Diego Ficarra, it is not possible to verify where he was born. Diego Ficarra could be a private individual with no publicly available information regarding his birthplace.)', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'T

FALSE
Percentage of True values: 30.0%
